# News Stream Analysis

This case study seeks to read a news RSS feed and process the headlines are they arrive!

To begin, uncomment the next cell and run it to install `feedparser` using pip. You need only do it once (but doing it more than once won't hurt anything). Run the next cell.

In [ ]:
# !pip install feedparser

This notebook depends on two external files: `news_feeder.py` and `moral_foundations.py`. They are in the same directory as the `.ipynb` file but your setup may be different and a bit of experimentation may be involved.

In [19]:
import news_feeder, moral_foundations

In [20]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/singhj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/singhj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [21]:
# Streaming Cell
import sys
for feed in news_feeder.feeds:
    for (tt, name, title, link) in feed.getHeadline():
        print (title)
        sys.stdout.flush()


First Coronavirus Vaccines Head to States, Starting Historic Effort
UPS and FedEx to Ship the First Vaccines on Sunday Morning
As Bids to Overturn Vote Fail, Pro-Trump Demonstrators Stick With Him
‘An Indelible Stain’: How the G.O.P. Tried to Topple a Pillar of Democracy
Republicans Find Themselves Speechless Following a Supreme Court Defeat
In Trying for a Diverse Administration, Biden Finds One Group’s Gain Is Another’s Loss
Trump Castigates Barr for Not Publicly Disclosing Hunter Biden Investigation
Federal Judge in Wisconsin Deals Trump Another Court Defeat
Cuomo Tries to Jolt Public by Warning of Overwhelmed Hospitals
Piano Bars and Jazz Clubs Reopened, Calling Live Music ‘Incidental’
Charley Pride, Country Music’s First Black Superstar, Dies at 86
A Guide to Georgia’s Senate Runoffs
Got a confidential news tip?
Movie Theaters Are Dying. Did Jason Kilar Deal the Final Blow?
Al Gore: I Have Hope on the Climate Crisis. America Must Lead.
The Texas Lawsuit and the Age of Dreampolitik

KeyboardInterrupt: 

In [22]:
import collections
samples = collections.deque(maxlen=100)

In [23]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = samples.maxlen
n_features = 1000
n_components = 8
n_top_words = 10

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

for feed in news_feeder.feeds:
    for (tt, name, title, link) in feed.getHeadline():
        samples.append(title)
        if len(samples) <= 50:
            # Collect 50 samples before starting to tokenize, etc.
            continue

        sent_text = nltk.sent_tokenize(title) # this gives us a list of sentences
        # now loop over each sentence and tokenize it separately
        for sentence in sent_text:
            tokenized_text = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(tokenized_text)
            print(tagged)
            lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                            learning_method='online',
                                            learning_offset=50.,
                                            random_state=0)
            # Use tf (raw term count) features for LDA.
            print("Extracting tf features for LDA...")
            tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                            max_features=n_features,
                                            stop_words='english')
            data_samples = list(samples)
            # data_samples = title[:n_samples]

            tf = tf_vectorizer.fit_transform(data_samples)
            
            lda.fit(tf)


            print("\nTopics in LDA model:")
            tf_feature_names = tf_vectorizer.get_feature_names()
            print_top_words(lda, tf_feature_names, n_top_words)

[('Stock', 'NNP'), ('Tank', 'NNP'), ('Pools', 'NNP'), ('for', 'IN'), ('Your', 'NNP'), ('Backyard', 'NNP')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: coronavirus group soccer loss vaccines age biden trump dies right
Topic #1: right coronavirus court music republicans hope theaters faced dies soccer
Topic #2: theaters republicans biden trump hope coronavirus vaccines loss covid court
Topic #3: dies music faced soccer age theaters vaccines court hope loss
Topic #4: vaccines music hope court faced biden covid dies loss republicans
Topic #5: covid biden faced republicans defeat vaccines theaters loss hope group
Topic #6: trump court republicans defeat faced loss biden coronavirus age dies
Topic #7: hope theaters soccer music trump biden loss faced court coronavirus

[('Spelling', 'VBG'), ('Bee', 'NNP')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: coronavirus soccer group loss vaccines age biden faced trump dies
Topic #1: right coronavirus court 

[('The', 'DT'), ('Virus', 'NNP'), ('Has', 'NNP'), ('Stolen', 'NNP'), ('Your', 'PRP$'), ('Face', 'NN'), ('From', 'IN'), ('Me', 'NNP')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: faced soccer 19 al kenya gore defeat group effects lawsuit
Topic #1: dreampolitik lawsuit age texas music won vaccine solve crisis al
Topic #2: group dies age biden loss texas crisis lead hurt republicans
Topic #3: coronavirus vaccine solve vaccines won gore al america court trump
Topic #4: covid defeat girls hurt court 19 effects social kenya trump
Topic #5: theaters biden vaccines faced crisis social defeat soccer effects solve
Topic #6: trump loss republicans coronavirus biden faced kenya lead dies music
Topic #7: hope lead gore right climate america dies al crisis soccer

[('A', 'DT'), ('Fake', 'NNP'), ('‘', 'NNP'), ('War', 'NNP'), ('on', 'IN'), ('Christmas', 'NNP'), ('’', 'NNP'), ('and', 'CC'), ('the', 'DT'), ('Real', 'JJ'), ('Battle', 'NNP'), ('Against', 'NNP'), ('Covid', 'NNP')]
Ext

[('128', 'CD'), ('Tricky', 'JJ'), ('Questions', 'NNS'), ('That', 'WDT'), ('Could', 'NNP'), ('Stand', 'NNP'), ('Between', 'NNP'), ('You', 'PRP'), ('and', 'CC'), ('U.S', 'NNP'), ('.', '.')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: covid 19 effects girls hurt social kenya al joe defeat
Topic #1: faced world loss play town soccer trump republicans lead coronavirus
Topic #2: stand like biden joe hurt right age dies group lead
Topic #3: hope lead climate coronavirus crisis vaccines al america gore soccer
Topic #4: war stolen trump covid music effects crisis play stand defeat
Topic #5: did age music dreampolitik dies town lawsuit theaters texas play
Topic #6: biden defeat won right vaccine group court trump coronavirus solve
Topic #7: trump covid world war right effects solve republicans gore coronavirus

[('Citizenship', 'NN')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: covid 19 effects girls hurt social kenya al joe defeat
Topic #1: faced worl

[('Pfizer', 'NNP'), ("'s", 'POS'), ('Covid', 'NNP'), ('Vaccine', 'NNP'), ('and', 'CC'), ('Allergies', 'NNS'), (':', ':'), ('How', 'NNP'), ('Concerned', 'NNP'), ('Should', 'NNP'), ('You', 'PRP'), ('Be', 'VB'), ('?', '.')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: covid 19 effects girls coronavirus won social world solve kenya
Topic #1: music dies stolen soccer age court gore covid defeat faced
Topic #2: biden gore hope climate crisis al lead vaccine pfizer america
Topic #3: pandemic court defeat republicans trump 19 war al stolen biden
Topic #4: joe biden effort vaccines play starting head historic states coronavirus
Topic #5: right texas dreampolitik lawsuit age coronavirus like group effort america
Topic #6: trump did theaters faced loss republicans america town states stolen
Topic #7: stand war covid women hope effects vaccine hurt trump defeat

[('The', 'DT'), ('Loneliest', 'JJS'), ('Childhood', 'NN'), (':', ':'), ('Toddlers', 'NNS'), ('Have', 'VBP'), ('No', 

[('F.D.A', 'NNP'), ('.', '.')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: covid 19 effects girls stolen social kenya hurt vaccine young
Topic #1: health theaters hard world covid biden did 19 texas vaccine
Topic #2: right pfizer music vaccine hard solve al racial young panel
Topic #3: coronavirus trump vaccinated solve vaccine need won court defeat war
Topic #4: biden pandemic age group covid texas stand loss dies lawsuit
Topic #5: vaccine america hope covid climate gore lead pfizer racial al
Topic #6: town solve theaters group world vaccines dreampolitik lawsuit right effort
Topic #7: vaccines starting like head play historic coronavirus states soccer effort

[('Panel', 'NNP'), ('Gives', 'VBZ'), ('Green', 'JJ'), ('Light', 'NNP'), ('to', 'TO'), ('Pfizer', 'NNP'), ("'s", 'POS'), ('Covid', 'NNP'), ('Vaccine', 'NNP')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: covid 19 effects girls stolen social kenya hurt vaccine young
Topic #1: health theat

...

[('Climate', 'NNP'), ('change', 'NN'), (':', ':'), ('Snowy', 'NNP'), ('UK', 'NNP'), ('winters', 'NNS'), ('could', 'MD'), ('become', 'VB'), ('thing', 'NN'), ('of', 'IN'), ('the', 'DT'), ('past', 'NN')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: climate change covid students coronavirus scottish carbon join uni uk
Topic #1: children covid city teacher 19 student history war liverpool tests
Topic #2: teacher big change emissions join uk climate brexit war carbon
Topic #3: loneliness grades advice lockdown cut home brexit labour johnson pandemic
Topic #4: university mayor liverpool children urged covid manchester pandemic finds boris
Topic #5: covid pupils wales secondary school 19 schools deprived coronavirus help
Topic #6: brexit new schools uk covid friends does close trade online
Topic #7: exams human things cancelled pupils schools covid home 2021 children

[('Astronomy', 'NN'), (':', ':'), ("'Odd", 'JJ'), ('radio', 'NN'), ('circles', 'NNS'), ("'", 'POS'), ('in'

[('China', 'NNP'), ("'s", 'POS'), ('Chang', 'NNP'), ("'", 'POS'), ('e-5', 'JJ'), ('mission', 'NN'), ('leaves', 'VBZ'), ('Moon', 'NNP'), ("'s", 'POS'), ('surface', 'NN')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: students advice war covid children loneliness world grades year history
Topic #1: join change uk did online coronavirus brexit talks kent pupils
Topic #2: cities coronavirus student war summer big living deal climate deprived
Topic #3: covid schools pupils 19 secondary school wales london children close
Topic #4: brexit teacher new city human friends old living half talks
Topic #5: lockdown university home coronavirus exams uni pandemic loneliness did student
Topic #6: climate change help mission children carbon year school scottish mum
Topic #7: climate change brexit deal 2021 uk johnson big likely help

[('Hayabusa-2', 'NN'), (':', ':'), ('Capsule', 'NN'), ('with', 'IN'), ('asteroid', 'JJ'), ('samples', 'NNS'), ('in', 'IN'), ("'perfect", 'NNP'), ("'", 

[('Earth', 'NNP'), ('Photo', 'NNP'), ('winners', 'NNS'), ('announced', 'VBD')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: new city coronavirus loneliness advice covid did lockdown students 19
Topic #1: schools covid close online mum rashford change things wales england
Topic #2: climate change cities help carbon join deprived uk history war
Topic #3: summer big student points war satellite home scottish coronavirus nasa
Topic #4: covid exams pupils university 2021 urged school secondary teacher 19
Topic #5: human grades set things living teacher need deprived half online
Topic #6: pupils tests covid ireland schools kent nasa change cut high
Topic #7: children shape new catch finds history nasa moon pandemic school

[('Leg-lengthening', 'NN'), (':', ':'), ('The', 'DT'), ('people', 'NNS'), ('having', 'VBG'), ('surgery', 'NN'), ('to', 'TO'), ('be', 'VB'), ('a', 'DT'), ('bit', 'NN'), ('taller', 'NN')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: 

[('Last', 'JJ'), ('of', 'IN'), ('Us', 'NNP'), ('2', 'CD'), ('wins', 'NNS'), ('big', 'JJ'), ('at', 'IN'), ('The', 'DT'), ('Game', 'NNP'), ('Awards', 'NNP')]
Extracting tf features for LDA...

Topics in LDA model:
Topic #0: advice set loneliness coronavirus year home lockdown tests nasa student
Topic #1: children teacher exams covid pupils city cancelled pandemic schools new
Topic #2: living uni students things coronavirus manchester help university student half
Topic #3: help mum new nasa rashford climate wales pupils manchester moon
Topic #4: mission china university chang given moon ai half online shape
Topic #5: change exams climate satellite big pupils tests summer high home
Topic #6: climate change war covid catch cut schools deprived students world
Topic #7: covid schools secondary 19 star big pupils close shape tests

[('Drone', 'NN'), ('completes', 'VBZ'), ('first', 'JJ'), ('freight', 'NN'), ('flight', 'NN'), ('to', 'TO'), ('Isles', 'NNP'), ('of', 'IN'), ('Scilly', 'NNP')]
Extra

KeyboardInterrupt: 

In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

## Moral Foundations Theory

<img align="right" style="padding-left:10px; height: 55%; width: 55%" src="figures/political-camps-moral-foundations.png" ></a>

The second part of this case study is Moral Foundations Theory. Google the phrase or visit [MoralFoundations.org](https://moralfoundations.org/) to learn more about the theory (or watch [Jonathan Haidt's Ted Talk](https://www.ted.com/talks/jonathan_haidt_the_moral_roots_of_liberals_and_conservatives/)).

For the purpose of this exercise, you needn't learn much about the theory &mdash; we will be using the Moral Foundations team's list of words that connote different dimensions of morality. 

* **Care** connotes safety, peace, compassion, etc.
* **Harm**, Care's opposite, connotes war, fight, hurt, kill, suffer, etc.

In the coding, the above two moral opposites are called 'HarmVirtue' and 'HarmVice' respectively. Similarly for other moral dimensions. The categories and the words that belong to each are available on-line in a [Moral Foundations Dictionary](https://moralfoundations.org/wp-content/uploads/files/downloads/moral%20foundations%20dictionary.dic). The dictionary words often include &ast;s, for example `peace`&ast;, which could be peace, peaceful, peacefully, even peacenik!

In the next cell, we retrieve the groups and codes. Go ahead and print them to discern the pattern.

In [ ]:
groups, codes = moral_foundations.read_dicts()

In [ ]:
groups

### Codes

Print the codes first

In [ ]:
codes

### Understanding Codes

The first key-value in the dictionary is `'safe*': '01'.` It means that safe, safely, safety, ... are all in 
group 01 (HarmVirtue). 

Also notice that some words (`preserve`, for example) belong in multiple groups.

### <span style="color:blue"> 2. Write a function for finding categories of a word </span>

We want a function `find_word_categories(word)` such that the **assert** statements at the end of the next cell all pass.

In [ ]:
def find_word_categories(word):
    # fill in
    return []

assert (find_word_categories('hurt') == ['02'])
assert (find_word_categories('tradition') == ['07'])
assert (find_word_categories('disease') == ['10'])
assert (find_word_categories('@nytnational') == [])
assert (find_word_categories('preserve') == ['01', '07', '09'])

### <span style="color:blue"> 3. Classify each headline with its moral foundation group </span>

Identify each headline with its moral code group. (Some headlines won't have any at all).

A headline like "A tradition can be many things — for some, it's food. For others, it's faith. For many, family. "
should find {'AuthorityVirtue': 'tradition'} and {'IngroupVirtue': 'family'}.

Rewrite the streaming cell to print the moral code group along with the title.

**Note:** There are many corner cases in this problem. Your answer will be evaluated on how cleanly and completely you address them.

# when you have completed this exercise

1. Rerun all cells from top to bottom and check that they work. Please sign the next line.

2. <span style="color:blue">I, **(your name)**, certify that the work in this notebook is my own. </span>
    
3. <span style="color:blue">Further, I promise not to share or discuss the solutions until after the submission period has passed (on midnight of 10/18/20). </span>

You can submit a notebook by saving it as PDF. In the cluster environment, it's File | Print (Save as PDF) and submit to Gradescope. https://www.gradescope.com/courses/182658,On other versions, it may be File | Download As (PDF) and then submit to Gradescope.

To submit to Gradescope, log into the [website](https://www.gradescope.com/courses/182658), add course **9W7PW3** (if not already added) and submit. The assignment name should match the name of this notebook.

# Good luck!